![MONAI Logo](monai.png)

# Creating a Simple Image Processing App with MONAI Deploy App SDK

This tutorial introduces the basic concepts of the MONAI Deploy App SDK by showing how to develop a simple image processing application.

First, you will need to design the workflow of your application that defines Operators (tasks) and flows among them. Once the workflow is designed, you can use existing operators provided by MONAI Deploy or start implementing your customer operator classes. Next, you'll implement an Application class to construct a workflow graph with the operators.

![MONAI Deploy App SDK Workflow](images/sdk_workflow.png)

You can execute and debug your application locally in a Jupyter notebook or CLI.

### Contents
* [Setup](#setup)
* [Creating Operator Class](#create_op)
* [Creating Application Class](#create_app)
* [Executing App Locally](#execute_app_local)
* [Conclusion](#conclusion)

## Creating Operators and connecting them in the Application class

We will implement an application that consists of three Operators:

- **SobelOperator**: Apply a Sobel edge detector.
    - **Input**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **MedianOperator**: Apply a Median filter for noise reduction.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
- **GaussianOperator**: Apply a Gaussian filter for smoothening.
    - **Input**: an image object in memory ([`Image`](/modules/_autosummary/monai.deploy.core.domain.Image))
    - **Output**: a file path ([`DataPath`](/modules/_autosummary/monai.deploy.core.domain.DataPath))
    
The workflow of the application would look like this.

![Application Workflow](images/app_workflow.png)

### Using Google Colab

This notebook has the pip command for installing MONAI and will be added to any subsequent notebook.

**Enabling GPU Support**

To use GPU resources through Colab, change the runtime to GPU:

1. From the **"Runtime"** menu select **"Change Runtime Type"**
2. Choose **"GPU"** from the drop-down menu
3. Click **"SAVE"**

This will reset the notebook and probably ask you if you are a robot (these instructions assume you are not)

### Verify GPU Access

Running **!nvidia-smi** in a cell will verify this has worked and show you what kind of hardware you have access to.    

In [ ]:
# Install necessary image loading/processing packages for the application
!pip install -q "Pillow"
!pip install -q "scikit-image"
!pip install -q "wget"
!pip install -q "pydicom"
!pip install -q "highdicom"
!pip install -q "matplotlib"
!pip install -q "typeguard==2.12.1"
%matplotlib inline

# Install MONAI Deploy App SDK package
!pip install -qU "monai-deploy-app-sdk"
!pip install -qU "monai[ignite, nibabel, torchvision, tqdm]==1.1.0"

### 1. Setup

To begin, check that the NVIDIA driver has been installed correctly. The `nvidia-smi` command should run and output information about the GPUs on your system:

In [ ]:
!nvidia-smi

### 1.1 Setup environment

We'll set up folder called notebook_0 where we'll extract our data, models, and write out output.

In [ ]:
NOTEBOOK_ROOT="notebook_0/"
!mkdir -p notebook_0

### 1.2 Download test input

We will use a test input from the following.

> Case courtesy of Dr Bruno Di Muzio, <a href="https://radiopaedia.org/?lang=us">Radiopaedia.org</a>. From the case <a href="https://radiopaedia.org/cases/41113?lang=us">rID: 41113</a>

In [ ]:
from skimage import io
import wget

test_input_path = NOTEBOOK_ROOT+"normal-brain-mri-4.png"
wget.download("https://user-images.githubusercontent.com/1928522/133383228-2357d62d-316c-46ad-af8a-359b56f25c87.png", out=test_input_path)

print(f"Test input file path: {test_input_path}")

test_image = io.imread(test_input_path)
io.imshow(test_image)

### 1.3 Setup imports

Let's import necessary classes/decorators to define Application and Operator.

In [ ]:
import monai.deploy.core as md  # 'md' stands for MONAI Deploy (or can use 'core' instead)
from monai.deploy.core import (
    Application,
    DataPath,
    ExecutionContext,
    Image,
    InputContext,
    IOType,
    Operator,
    OutputContext,
)

<a id='create_op'></a>
## 2. Creating Operator classes

Each Operator class inherits [Operator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html) class and input/output properties are specified by using [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html) decorators.

Note that the first operator(SobelOperator)'s input and the last operator(GaussianOperator)'s output are [DataPath](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html) type with [IOType.DISK](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.IOType.html). Those paths are mapped into input and output paths given by the user during the execution.

Business logic would be implemented in the <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html#monai.deploy.core.Operator.compute">compute()</a> method.

### 2.1 SobelOperator

SobelOperator is the first operator (A root operator in the workflow graph). <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.InputContext.html#monai.deploy.core.InputContext.get">op_input.get(label)</a> (since only one input is defined in this operator, we don't need to specify an input label) would return an object of [DataPath](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html) and the input file/folder path would be available by accessing the `path` property (`op_input.get().path`).

Once an image data (as a Numpy array) is loaded and processed, [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object is created from the image data and set to the output (<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>).

In [ ]:
@md.input("image", DataPath, IOType.DISK)
@md.output("image", Image, IOType.IN_MEMORY)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class SobelOperator(Operator):
    """This Operator implements a Sobel edge detector.

    It has a single input and single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage import filters, io

        input_path = op_input.get().path
        if input_path.is_dir():
            input_path = next(input_path.glob("*.*"))  # take the first file

        data_in = io.imread(input_path)[:, :, :3]  # discard alpha channel if exists
        data_out = filters.sobel(data_in)

        op_output.set(Image(data_out))

### 2.2 MedianOperator

MedianOperator is a middle operator that accepts data from SobelOperator and passes the processed image data to GaussianOperator.

Its input and output data types are [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) and the Numpy array data is available through `asnumpy()` method (`op_input.get().asnumpy()`).

Again, once an image data (as a Numpy array) is loaded and processed, [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) object is created and set to the output (<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.OutputContext.html#monai.deploy.core.OutputContext.set">op_output.set(value, label)</a>).

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", Image, IOType.IN_MEMORY)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class MedianOperator(Operator):
    """This Operator implements a noise reduction.

    The algorithm is based on the median operator.
    It ingests a single input and provides a single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import median

        data_in = op_input.get().asnumpy()
        data_out = median(data_in)
        op_output.set(Image(data_out))

### 2.3 GaussianOperator

GaussianOperator is the last operator (A leaf operator in the workflow graph) and the output path of this operator is mapped to the user-provided output folder so we cannot set a path to `op_output` variable (e.g., `op_output.set(Image(data_out))`).

Instead, we can get the output path through `op_output.get().path` and save the processed image data into a file.

In [ ]:
@md.input("image", Image, IOType.IN_MEMORY)
@md.output("image", DataPath, IOType.DISK)
# If `pip_packages` is specified, the definition will be aggregated with the package dependency list of other
# operators and the application in packaging time.
# @md.env(pip_packages=["scikit-image >= 0.17.2"])
class GaussianOperator(Operator):
    """This Operator implements a smoothening based on Gaussian.

    It ingests a single input and provides a single output.
    """

    def compute(self, op_input: InputContext, op_output: OutputContext, context: ExecutionContext):
        from skimage.filters import gaussian
        from skimage.io import imsave

        data_in = op_input.get().asnumpy()
        data_out = gaussian(data_in, sigma=0.2)

        output_folder = op_output.get().path
        output_path = output_folder / "final_output.png"
        imsave(output_path, data_out)

<a id='create_app'></a>
## 3. Creating Application class

Our application class would look like below.

It defines `App` class, inheriting [Application](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html) class.

The requirements (resource and package dependency) for the App can be specified by using [@resource](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.resource.html) and [@env](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.env.html) decorators.

The @resource and @env decorators are also helpful from a Deployment IT / Infrastructure perspective in that they help with reproducibility and infrastructure requirements directly in the code.

In [ ]:
@md.resource(cpu=1)
# pip_packages can be a string that is a path(str) to requirements.txt file or a list of packages.
@md.env(pip_packages=["scikit-image >= 0.17.2"])
class App(Application):
    """This is a very basic application.

    This showcases the MONAI Deploy application framework.
    """

    # App's name. <class name>('App') if not specified.
    name = "simple_imaging_app"
    # App's description. <class docstring> if not specified.
    description = "This is a very simple application."
    # App's version. <git version tag> or '0.0.0' if not specified.
    version = "0.1.0"

    def compose(self):
        """This application has three operators.

        Each operator has a single input and a single output port.
        Each operator performs some kind of image processing function.
        """
        sobel_op = SobelOperator()
        median_op = MedianOperator()
        gaussian_op = GaussianOperator()

        self.add_flow(sobel_op, median_op)
        # self.add_flow(sobel_op, median_op, {"image": "image"})
        # self.add_flow(sobel_op, median_op, {"image": {"image"}})

        self.add_flow(median_op, gaussian_op)

In `compose()` method, objects of `SobelOperator`, `MedianOperator`, and `GaussianOperator` classes are created
and connected through <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a>.

> add_flow(upstream_op, downstream_op, io_map=None)

`io_map` is a dictionary of mapping from the source operator's label to the destination operator's label(s) and its type is `Dict[str, str|Set[str]]`. 

We can skip specifying `io_map` if both the number of `upstream_op`'s outputs and the number of `downstream_op`'s inputs are one so `self.add_flow(sobel_op, median_op)` is same with `self.add_flow(sobel_op, median_op, {"image": "image"})` or `self.add_flow(sobel_op, median_op, {"image": {"image"}})`.


<a id='execute_app_local'></a>
## 4. Executing app locally

Once an Application class (App) is defined, you can instantiate the Application and execute with the Application.run() method.

Utilizing Jupyter Notebooks with MONAI Deploy App SDK gives developers an easy way to get started building AI applications.  You have the full suite of Jupyter tools for debugging and fast iteration and then can quickly transition to creating a full-fledged AI Application docker container using the same tools.

In [ ]:
app = App()
app.run(input=test_input_path, output=NOTEBOOK_ROOT)

### 4.1 Show results 

We'll then use Matplotlib to show the output result.

In [ ]:
output_image = io.imread(NOTEBOOK_ROOT+"final_output.png")

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 15 ,15
fig, ax = plt.subplots(1,2)

ax[0].imshow(test_image);
ax[1].imshow(output_image);

<a id='conclusion'></a>
## Conclusion

In this notebook, we have walked through the process to create a simple application using MONAI Deploy App SDK and running the application in Jupyter.

### What's Next
The following notebook we'll walk through a more realistic classification task using the MedNIST dataset.